In [ ]:
%pip install langchain langchain-openai

In [7]:
%pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


True

In [32]:
from langsmith import Client as LangSmithClient

DATASET_NAME='Config mgmt Rap Battle Dataset'
client = LangSmithClient()
datasets=client.list_datasets()
for dataset in datasets:
    if (dataset.name) == DATASET_NAME:
        client.delete_dataset(dataset_name=DATASET_NAME)
        print(dataset)

In [33]:

# Inputs are provided to your model, so it know what to generate
dataset_inputs = [
    "a rap battle between Chef and Puppet",
    "a rap battle between Ansible and Pulumi",
    # ... add more as desired
]


In [34]:


# Storing inputs in a dataset lets us
# run chains and LLMs over a shared set of examples.
dataset = client.create_dataset(
    dataset_name=DATASET_NAME,
    description="Rap battle prompts.",
)
client.create_examples(
    inputs=[{"question": q} for q in dataset_inputs],
#    outputs=dataset_outputs,
    dataset_id=dataset.id,
)


In [37]:
from langchain import prompts
from langchain.schema import output_parser

# Define your runnable or chain below.
prompt = prompts.ChatPromptTemplate.from_messages(
    [("system", "You are a helpful AI assistant."), ("human", "{question}")]
)

from langchain_openai import ChatOpenAI
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

chain = prompt | chat | output_parser.StrOutputParser()
answer=chain.invoke({"question":"A rap battle between Kris and Toshaan"})
print(answer)

Ladies and gentlemen, gather 'round,
It's time for a rap battle showdown.
In this corner, we have Kris, the lyrical king,
And in the other corner, Toshaan, ready to bring.

Kris, you're up first, show 'em what you got,
Drop those rhymes, make 'em hot.
Your flow is smooth, your words are tight,
Let's see if Toshaan can handle your might.

Kris:
I'm the rap game's MVP, no doubt,
My rhymes hit hard, they knock you out.
I'm the king of the mic, the ruler of the beat,
Toshaan, you better take a seat.

I spit fire, my words burn like flames,
Leaving you in ashes, no one remembers your name.
I'm the rap god, the one they all admire,
Toshaan, you're just a small-time liar.

Toshaan, it's your turn, let's see what you got,
Can you match Kris, or will you be forgot?

Toshaan:
I may be small, but my rhymes are mighty,
I'll take you down, Kris, no need to be flighty.
I may not have the fame, but I got the skill,
I'll show you what it means to be real.

I'm the underdog, the one they underestimate,

In [39]:
from langsmith.evaluation import EvaluationResult, run_evaluator
from langsmith.schemas import Example, Run

@run_evaluator
def is_empty(run: Run, example: Example | None = None):
    model_outputs = run.outputs["output"]
    score = not model_outputs.strip()
    return EvaluationResult(key="is_empty", score=score)


from langchain.smith import RunEvalConfig, run_on_dataset

# Define the evaluators to apply
verifier_llm=chat
eval_config = RunEvalConfig(
    evaluators=[
        # You can define an arbitrary criterion as a key: value pair in the criteria dict
        RunEvalConfig.Criteria(
            {"creativity": "Is this submission creative, imaginative, or novel?"}
        ),
        # We provide some simple default criteria like "conciseness" you can use as well
        RunEvalConfig.Criteria("conciseness"),
        # "cot_qa",
        #        smith.RunEvalConfig.LabeledCriteria("conciseness"),
        #        smith.RunEvalConfig.LabeledCriteria("relevance"),
        #        smith.RunEvalConfig.LabeledCriteria("coherence"),
        #        smith.RunEvalConfig.LabeledCriteria("harmfulness"),
        #        smith.RunEvalConfig.LabeledCriteria("insensitivity"),
        #        smith.RunEvalConfig.LabeledCriteria("criminality"),
        #        smith.RunEvalConfig.LabeledCriteria("misogyny"),
        #        smith.RunEvalConfig.LabeledCriteria("controversiality"),
        #        smith.RunEvalConfig.LabeledCriteria("helpfulness"),
        #        smith.RunEvalConfig.LabeledCriteria("maliciousness"),
    ],
    custom_evaluators=[is_empty],
    eval_llm=verifier_llm,
)

chain_results = client.run_on_dataset(
    dataset_name=DATASET_NAME,
    llm_or_chain_factory=chain,
    evaluation=eval_config,
    concurrency_level=5,
    verbose=True,
)


View the evaluation results for project 'advanced-tray-56' at:
https://smith.langchain.com/o/27713559-0b06-4e07-89cf-5339d432c5c3/datasets/16931063-e62f-484c-96f5-2a2701aeb7ce/compare?selectedSessions=37a69093-636e-4b29-b71a-c1652ab88019

View all tests for Dataset Config mgmt Rap Battle Dataset at:
https://smith.langchain.com/o/27713559-0b06-4e07-89cf-5339d432c5c3/datasets/16931063-e62f-484c-96f5-2a2701aeb7ce
[------------------------------------------------->] 2/2

In [44]:
from pprint import pprint 

pprint(chain_results)

{'project_name': 'advanced-tray-56',
 'results': {'880ce2e3-361d-43d5-8bb2-2831ecd05c42': {'execution_time': 6.787325,
                                                      'feedback': [EvaluationResult(key='creativity', score=1, value='Y', comment='To determine if the submission meets the criteria of creativity, we need to assess if it is creative, imaginative, or novel. Here is the step-by-step reasoning:\n\n1. The submission presents a rap battle between Chef and Puppet, which is a unique and imaginative concept.\n2. The lyrics of the rap battle showcase creativity in the use of rhymes, wordplay, and imagery.\n3. The submission introduces a playful and entertaining tone, adding to its creativity.\n\nBased on the above reasoning, the submission meets the criteria of creativity.', correction=None, evaluator_info={'__run': RunInfo(run_id=UUID('010dcb04-4a40-409f-82ae-9ed0d1012e53'))}, source_run_id=None, target_run_id=None),
                                                             